<a href="https://colab.research.google.com/github/SRARNAB7/HDS_5230_07_Arnab/blob/main/Week%2009/Week_09_Assignment_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NAME- ARNAB SUTAR
# ASSIGNMENT- WEEK 09
**Week 09 - Machine Learning with Scikit-learn**

**1) Among the different classification models included in the Python notebook, which model had the best overall performance? Support your response by referencing appropriate evidence.**

In [1]:
## Import Modules
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.metrics import confusion_matrix
import sklearn
from sklearn import datasets

In [2]:
## Set default figure size to be larger
## this may only work in matplotlib 2.0+!
matplotlib.rcParams['figure.figsize'] = [10.0,6.0]
## Enable multiple outputs from jupyter cells
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
## Get Version information
print(sys.version)
print("Pandas version: {0}".format(pd.__version__))
print("Matplotlib version: {0}".format(matplotlib.__version__))
print("Numpy version: {0}".format(np.__version__))
print("SciKitLearn version: {0}".format(sklearn.__version__))

3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Pandas version: 2.2.2
Matplotlib version: 3.10.0
Numpy version: 2.0.2
SciKitLearn version: 1.6.1


Check the working directory Set the working directory to make paths easier :)

In [4]:
# Working Directory
import os
print("My working directory:\n" + os.getcwd())
# Set Working Directory
os.chdir(".")
print("My new working directory:\n" + os.getcwd())

My working directory:
/content
My new working directory:
/content


Patient Mortality Dataset We will use a dataset with a binary outcome of mortality as a motivating example.

This is a dataset of patients demographics and disease status, with mortality indicated. The dataset is here:

data\healthcare\patientAnalyticFile.csv

In practice, you most likely would have created a dataset like this from multiple other files after cleaning, reshaping, and joining them.

You can generalize this setup to any situation with a binary outcome, such as estimating the probability of a customer filing a warranty claim, or the probability of a transaction being fraudulent.

We will first import this dataset and examine the potential variables to use in our classification algorithm.

In [5]:
## Set print limits
pd.options.display.max_rows = 10
## Import Data
df_patient = \
 pd.read_csv('./PatientAnalyticFile.csv')
df_patient

,PatientID,DateOfBirth,Gender,Race,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,...,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,First_Appointment_Date,Last_Appointment_Date,DateOfDeath
0,1,1962-02-27,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2013-04-27,2018-06-01,NaN
1,2,1959-08-18,male,white,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2005-11-30,2008-11-02,2008-11-02
2,3,1946-02-15,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2011-11-05,2015-11-13,NaN
3,4,1979-07-27,female,white,0,0,0,0,0,1,...,0,0,0,0,0,0,0,2010-03-01,2016-01-17,2016-01-17
4,5,1983-02-19,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2006-09-22,2018-06-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,1997-12-19,female,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2008-06-14,2018-06-01,NaN
19996,19997,1984-03-31,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2007-04-24,2018-06-01,NaN
19997,19998,1993-07-04,female,white,0,0,0,0,0,0,...,0,0,1,0,1,0,0,2010-10-16,2018-06-01,NaN
19998,19999,1984-04-17,male,other,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2015-01-04,2018-06-01,NaN


We need to make a variable to indicate mortality. We can do that based on the abscence of 'date of death':

In [6]:
# Create mortality variable
df_patient['mortality'] = \
    np.where(df_patient['DateOfDeath'].isnull(),
             0,1)
# Examine
df_patient['mortality']

,mortality
0,0
1,1
2,0
3,1
4,0
...,...
19995,0
19996,0
19997,0
19998,0


In [7]:
df_patient['mortality'].describe()

,mortality
count,20000.000000
mean,0.354700
std,0.478434
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [8]:
df_patient.describe()

,PatientID,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,Rheumatic,Peptic_ulcer_disease,LiverMild,...,Cancer,LiverSevere,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,mortality
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,0.045600,0.043450,0.023950,0.028650,0.031400,0.072650,0.012300,0.009650,0.009250,...,0.050450,0.051450,0.033150,0.006450,0.163450,0.106300,0.302900,0.040050,0.079750,0.354700
std,5773.647028,0.208621,0.203873,0.152897,0.166825,0.174401,0.259568,0.110224,0.097762,0.095733,...,0.218877,0.220919,0.179033,0.080054,0.369785,0.308229,0.459524,0.196081,0.270913,0.478434
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5000.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10000.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,15000.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,20000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
df_patient.dtypes

,0
PatientID,int64
DateOfBirth,object
Gender,object
Race,object
Myocardial_infarction,int64
...,...
Alcohol,int64
First_Appointment_Date,object
Last_Appointment_Date,object
DateOfDeath,object


We should change date of birth to be an actual date and calculate age if we want to include it in the model:

In [10]:
# Convert dateofBirth to date
df_patient['DateOfBirth'] = \
    pd.to_datetime(df_patient['DateOfBirth'])
# Calculate age in years as of 2015-01-01
df_patient['Age_years'] = \
    ((pd.to_datetime('2015-01-01') - df_patient['DateOfBirth']).dt.days/365.25)
df_patient['Age_years'].describe()

,Age_years
count,20000.000000
mean,47.247474
std,18.145086
min,15.753593
25%,31.733744
50%,47.099247
75%,62.924025
max,78.743326


Use Patsy to Create the Model Matrices We typically start out with a pandas dataframe for manipulation purposes, then we will use this dataframe as the input to the machine learning library. I created a pandas dataframe above to replicate this process. We will use the dmatrices function from the patsy library to easily generate the design matrices for the machine learning algorithms representing the inputs. This handles the following:

drops rows with missing data construct one-hot encoding for categorical variables optionally adds constant intecercept

In [11]:
df_patient.columns

Index(['PatientID', 'DateOfBirth', 'Gender', 'Race', 'Myocardial_infarction',
       'Congestive_heart_failure', 'Peripheral_vascular_disease', 'Stroke',
       'Dementia', 'Pulmonary', 'Rheumatic', 'Peptic_ulcer_disease',
       'LiverMild', 'Diabetes_without_complications',
       'Diabetes_with_complications', 'Paralysis', 'Renal', 'Cancer',
       'LiverSevere', 'Metastatic_solid_tumour', 'HIV', 'Obesity',
       'Depression', 'Hypertension', 'Drugs', 'Alcohol',
       'First_Appointment_Date', 'Last_Appointment_Date', 'DateOfDeath',
       'mortality', 'Age_years'],
      dtype='object')

In [12]:
## Create formula for all variables in model
vars_remove = ['PatientID','First_Appointment_Date','DateOfBirth',
               'Last_Appointment_Date','DateOfDeath','mortality']
vars_left = set(df_patient.columns) - set(vars_remove)
formula = "mortality ~ " + " + ".join(vars_left)
formula

'mortality ~ Race + Stroke + Myocardial_infarction + Diabetes_with_complications + Paralysis + Gender + LiverSevere + Rheumatic + LiverMild + Metastatic_solid_tumour + Peptic_ulcer_disease + Depression + Age_years + Diabetes_without_complications + HIV + Alcohol + Obesity + Congestive_heart_failure + Dementia + Hypertension + Peripheral_vascular_disease + Pulmonary + Cancer + Drugs + Renal'

In [13]:
## only use subset of data so models fit in reasonable time
df_patient_sub = \
    df_patient.sample(frac=0.1,
                     random_state=32)
## use Patsy to create model matrices
Y,X = dmatrices(formula,
                df_patient_sub)

In [14]:
Y

DesignMatrix with shape (2000, 1)
  mortality
          0
          0
          1
          1
          0
          0
          1
          1
          0
          0
          1
          0
          1
          0
          1
          0
          1
          0
          0
          1
          0
          1
          0
          0
          0
          0
          1
          1
          0
          0
  [1970 rows omitted]
  Terms:
    'mortality' (column 0)
  (to view full data, use np.asarray(this_obj))

In [15]:
X

DesignMatrix with shape (2000, 28)
  Columns:
    ['Intercept',
     'Race[T.hispanic]',
     'Race[T.other]',
     'Race[T.white]',
     'Gender[T.male]',
     'Stroke',
     'Myocardial_infarction',
     'Diabetes_with_complications',
     'Paralysis',
     'LiverSevere',
     'Rheumatic',
     'LiverMild',
     'Metastatic_solid_tumour',
     'Peptic_ulcer_disease',
     'Depression',
     'Age_years',
     'Diabetes_without_complications',
     'HIV',
     'Alcohol',
     'Obesity',
     'Congestive_heart_failure',
     'Dementia',
     'Hypertension',
     'Peripheral_vascular_disease',
     'Pulmonary',
     'Cancer',
     'Drugs',
     'Renal']
  Terms:
    'Intercept' (column 0)
    'Race' (columns 1:4)
    'Gender' (column 4)
    'Stroke' (column 5)
    'Myocardial_infarction' (column 6)
    'Diabetes_with_complications' (column 7)
    'Paralysis' (column 8)
    'LiverSevere' (column 9)
    'Rheumatic' (column 10)
    'LiverMild' (column 11)
    'Metastatic_solid_tumour' (colu


**Split into Testing and Training Sample**

In [16]:
## Split Data into training and sample
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X,
                     np.ravel(Y), # prevents dimensionality error later!
                     test_size=0.25,
                     random_state=42)

In [17]:
## Confirm dimensions
X_train.shape

(1500, 28)

In [18]:
X_test.shape

(500, 28)

In [19]:
y_train.shape

(1500,)

In [20]:
y_test.shape

(500,)

**First Model: Logistic Regression**

In [21]:
## import linear model
from sklearn import linear_model
## Define model parameters
## can implement penalties, but check docs for appropriate solver
clf = linear_model.LogisticRegression(fit_intercept=True, # already have the intercept
                                      solver='liblinear') # could change to lbfgs!
## fit model using data with .fit
clf.fit(X_train,y_train)

LogisticRegression(solver='liblinear')

In [22]:
## Make predictions on training dataset
## training error?
clf.predict(X_train)

array([0., 0., 0., ..., 0., 0., 0.])

In [23]:
## get mean accuracy
clf.score(X_train,y_train)

0.7333333333333333

**Keep Track of Scores Across Models**

In [24]:
## Create dict to store all these results:
result_scores = {}
## Score the Model on Training and Testing Set
result_scores['Logistic'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [25]:
## Create Function to Print Results
def get_results(x1):
    print("\n{0:20}   {1:4}    {2:4}".format('Model','Train','Test'))
    print('-------------------------------------------')
    for i in x1.keys():
        print("{0:20}   {1:<6.4}   {2:<6.4}".format(i,x1[i][0],x1[i][1]))

In [26]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 


**Comparison to the Null Model**

In [27]:
## Null information rate
1 - y_train.mean()

array(0.64666667)

In [28]:
## Dummy classifier
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent',
                      random_state=0)
clf.fit(X_train, y_train)
clf.score(X_train, y_train)

DummyClassifier(random_state=0, strategy='most_frequent')

0.6466666666666666

In [30]:
## Score the Model on Training and Testing Set
result_scores['Null'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [ ]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 


**Regularized Linear Regression**

In [31]:
## Logistic Regression with l1 penalty
## Specify penalty directly as C = 1
clf = linear_model.LogisticRegression(penalty='l1',
                                      C=1, solver = 'liblinear') # specify penalty
clf.fit(X_train,y_train)
## get confusion matrix
confusion_matrix(y_train,clf.predict(X_train))

LogisticRegression(C=1, penalty='l1', solver='liblinear')

array([[820, 150],
       [252, 278]])

In [32]:
## get accuracy
sklearn.metrics.accuracy_score(y_train,clf.predict(X_train))

0.732

In [33]:
## Score the Model on Training and Testing Set
result_scores['Logistic_L1_C_1'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 


In [34]:
## Logistic Regression with l1 penalty
## Specify penalty directly as C = 0.1
clf = linear_model.LogisticRegression(penalty='l1',
                                      C=0.1, solver = 'liblinear') # specify penalty
clf.fit(X_train,y_train)
## Score the Model on Training and Testing Set
result_scores['Logistic_L1_C_01'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)

LogisticRegression(C=0.1, penalty='l1', solver='liblinear')


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 


In [35]:
## Logistic Regression with l1 penalty
## Specify penalty directly as C = 0.1
clf = linear_model.LogisticRegression(penalty='l1',
                                      C=10, solver = 'liblinear') # specify penalty
clf.fit(X_train,y_train)
## Score the Model on Training and Testing Set
result_scores['Logistic_L1_C_10'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)

LogisticRegression(C=10, penalty='l1', solver='liblinear')


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 


**Selecting Parameters via Cross Validation**

In [36]:
## Select the alpha through cross validation (k-folds leave one out)
# auto generate 20 values between 1e-4 and 1e4 on log scale
clf = linear_model.LogisticRegressionCV(cv=5,
                                        Cs=20, ## takes awhile to fit 20 models!
                                        penalty='l1',
                                        solver='liblinear')
clf.fit(X_train,y_train)

LogisticRegressionCV(Cs=20, cv=5, penalty='l1', solver='liblinear')

In [37]:
## Score the Model on Training and Testing Set
result_scores['Logistic_L1_C_auto'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 


**Scaling / Pipeline**

In [38]:
## LASSO (L1) regression, set alpha
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
## set our transformation
scaler = preprocessing.StandardScaler()

## set our model
clf = linear_model.LogisticRegressionCV(cv=5,
                                        Cs=20, ## takes awhile to fit 20 models!
                                        penalty='l1',
                                        solver='liblinear')
## put together in pipeline
pipe1 = Pipeline([("scale", scaler),
                  ("LASSO", clf)])
pipe1.fit(X_train,y_train)

Pipeline(steps=[('scale', StandardScaler()),
                ('LASSO',
                 LogisticRegressionCV(Cs=20, cv=5, penalty='l1',
                                      solver='liblinear'))])

In [39]:
pipe1.named_steps

{'scale': StandardScaler(),
 'LASSO': LogisticRegressionCV(Cs=20, cv=5, penalty='l1', solver='liblinear')}

In [40]:
## Score the Model on Training and Testing Set
result_scores['Logistic_SL1_C_auto'] = \
            (sklearn.metrics.accuracy_score(y_train,pipe1.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,pipe1.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 
Logistic_SL1_C_auto    0.7307   0.714 


**Random forest**

In [41]:
#### Fit Random Forest
## Random Forests
from sklearn import ensemble
clf = ensemble.RandomForestClassifier(n_estimators=100,
                                      max_features=10,
                                      random_state=42)
clf.fit(X_train,y_train)
## get confusion matrix
confusion_matrix(y_train,clf.predict(X_train))

RandomForestClassifier(max_features=10, random_state=42)

array([[970,   0],
       [  1, 529]])

In [42]:
## Score the Model on Training and Testing Set
result_scores['RandomForest_noCV'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 
Logistic_SL1_C_auto    0.7307   0.714 
RandomForest_noCV      0.9993   0.682 


Based on the results presented in the screenshot, the Logistic regression model with L1 penalty and C=10 (Logistic_L1_C_10) demonstrated the best overall performance. It achieved the highest test accuracy of 0.718, which is a key indicator of how well the model generalizes to new, unseen data. In contrast, the Random Forest (no CV) model had an almost perfect training accuracy of 0.9993, but its test accuracy dropped sharply to 0.686, suggesting significant overfitting.

The Logistic_L1_C_10 model showed a strong balance between training and testing performance, with a training accuracy of 0.7347 and a test accuracy of 0.718. This close alignment indicates that the model generalizes well without being overly fitted to the training data. Therefore, among all the classification models evaluated, Logistic_L1_C_10 proved to be the most reliable and generalizable.

**2) Next, fitting a series of logistic regression models, without regularization. Each model should use the same set of predictors (all of the relevant predictors in the dataset) and should use the entire dataset, rather than a fraction of it. Use a randomly chosen 80% proportion of observations for training and the remaining for checking the generalizable performance (i.e., performance on the holdout subset). Be sure to ensure that the training and holdout subsets are identical across all models. Each model should choose a different solver.**

In [ ]:
Y, X = dmatrices(formula, df_patient)

In [ ]:
Y

DesignMatrix with shape (20000, 1)
  mortality
          0
          1
          0
          1
          0
          0
          0
          0
          0
          0
          0
          0
          1
          0
          0
          0
          1
          1
          0
          0
          1
          1
          0
          0
          0
          0
          0
          0
          0
          0
  [19970 rows omitted]
  Terms:
    'mortality' (column 0)
  (to view full data, use np.asarray(this_obj))

In [ ]:
X

DesignMatrix with shape (20000, 28)
  Columns:
    ['Intercept',
     'Gender[T.male]',
     'Race[T.hispanic]',
     'Race[T.other]',
     'Race[T.white]',
     'Pulmonary',
     'LiverSevere',
     'Congestive_heart_failure',
     'LiverMild',
     'Rheumatic',
     'Renal',
     'Paralysis',
     'HIV',
     'Cancer',
     'Age_years',
     'Peripheral_vascular_disease',
     'Obesity',
     'Depression',
     'Diabetes_without_complications',
     'Myocardial_infarction',
     'Metastatic_solid_tumour',
     'Peptic_ulcer_disease',
     'Diabetes_with_complications',
     'Stroke',
     'Drugs',
     'Alcohol',
     'Hypertension',
     'Dementia']
  Terms:
    'Intercept' (column 0)
    'Gender' (column 1)
    'Race' (columns 2:5)
    'Pulmonary' (column 5)
    'LiverSevere' (column 6)
    'Congestive_heart_failure' (column 7)
    'LiverMild' (column 8)
    'Rheumatic' (column 9)
    'Renal' (column 10)
    'Paralysis' (column 11)
    'HIV' (column 12)
    'Cancer' (column 13)
   

**Splitting into Holdout(Test) and Training Samples**

The below code is used to split into Traning and Test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, np.ravel(Y), test_size=0.2, random_state=42)

Confirming the Output Dimensions. The dimensions of the data are the same within test and train. The proportion should also be close to the test_size argument.

In [ ]:
## Confirm dimensions
X_train.shape

(16000, 28)

In [ ]:
X_test.shape

(4000, 28)

In [ ]:
y_train.shape

(16000,)

In [ ]:
y_test.shape

(4000,)

**Logistic Regression Models with Different Solvers that are not generalized. The different solvers are 'lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga'**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time
import pandas as pd

# List of solvers that support penalty=None
solvers = ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga']

# Storing the results
results = []

# Looping through solvers
for solver in solvers:
    print(f"\nTraining with solver = {solver}")

    # Initializing the model
    clf = LogisticRegression(
        penalty=None,
        solver=solver,
        fit_intercept=True,
        max_iter=1000,
        random_state=42
    )

    # Calculation of time of the fitting process
    start_time = time.time()
    clf.fit(X_train, y_train)
    end_time = time.time()

    # Evaluate
    train_acc = accuracy_score(y_train, clf.predict(X_train))
    test_acc = accuracy_score(y_test, clf.predict(X_test))
    duration = end_time - start_time

    # Storing the results
    results.append({
        "Solver Used": solver,
        "Training subset accuracy": round(train_acc, 4),
        "Holdout subset accuracy": round(test_acc, 4),
        "Time taken (seconds)": round(duration, 4)
    })

# Converting to DataFrame
results_df = pd.DataFrame(results)




Training with solver = lbfgs


LogisticRegression(max_iter=1000, penalty=None, random_state=42)


Training with solver = newton-cg


LogisticRegression(max_iter=1000, penalty=None, random_state=42,
                   solver='newton-cg')


Training with solver = newton-cholesky


...

LogisticRegression(max_iter=1000, penalty=None, random_state=42,
                   solver='newton-cholesky')


Training with solver = sag


LogisticRegression(max_iter=1000, penalty=None, random_state=42, solver='sag')


Training with solver = saga


LogisticRegression(max_iter=1000, penalty=None, random_state=42, solver='saga')

**3)Compare the results of the models in terms of their accuracy (use this as the performance metric to assess generalizability error on the holdout subset) and the time taken (use appropriate timing function). Summarize your results via a table with the following structure:**

In [ ]:
# Displaying the results in the table
results_df.sort_values(by="Holdout subset accuracy", ascending=False, inplace=True)
results_df.reset_index(drop=True, inplace=True)
results_df

,Solver Used,Training subset accuracy,Holdout subset accuracy,Time taken (seconds)
0,saga,0.7480,0.7360,3.4000
1,lbfgs,0.7481,0.7358,0.2055
2,newton-cholesky,0.7479,0.7358,0.2900
3,sag,0.7479,0.7358,2.9506
4,newton-cg,0.7481,0.7355,0.0686


**4) Based on the results, which solver yielded the best results? Explain the basis for ranking the models - did you use training subset accuracy? Holdout subset accuracy? Time of execution? All three? Some combination of the three?**

Based on the combination of test accuracy, training accuracy, and execution time, newton-cg is the best overall solver. While saga achieved the highest holdout (test) accuracy at 0.7360, the difference compared to newton-cg (0.7355) is extremely small — only 0.0005 — and practically insignificant. On the other hand, newton-cg completed training in just 0.0686 seconds, making it the fastest among all solvers while still maintaining strong accuracy and consistency.

lbfgs ranks second. It delivered the same holdout accuracy (0.7358) as newton-cholesky, but did so faster (0.2055 seconds vs 0.2900 seconds), making it a more efficient choice. newton-cholesky comes in third; it also offered competitive performance, but took slightly longer to run than lbfgs, making it slightly less optimal when time is a concern.

sag is ranked fourth. While its accuracy (0.7358) is nearly identical to the others, its execution time (2.9506 seconds) is much longer, which can limit its usefulness, especially for large datasets or repeated model training.

Finally, saga, despite achieving the highest test accuracy (0.7360), required the longest training time (3.4000 seconds). Given the marginal improvement in accuracy and the significant increase in computation time, it is not the most efficient choice for most real-world applications.

In conclusion, newton-cg is the best solver overall due to its excellent balance of accuracy, speed, and generalizability, making it the most practical and efficient choice based on the results shown.